### Import useful libraries

In [1]:
%run "Imports.ipynb"
%run "Helpers.ipynb"

### Load and prepare the data

In [2]:
# Raw data - position of each driver log
path_raw = 'supply_raw_data.csv'
df_raw = pd.read_csv(path_raw, parse_dates=['timestamp'])
df_raw['timestamp'].max()

Timestamp('2020-02-04 23:00:27')

In [3]:
# Supply per zone
path_supply = 'supply_per_zone_2.csv'
df_supply = pd.read_csv(path_supply, parse_dates=['timestamp'])
df_supply.head()

,Unnamed: 0,zone,timestamp,supply,forecast
0,0,0_0_10_10,2019-01-04 14:00:00,1,NaN
1,1,0_0_10_10,2019-01-04 15:00:00,0,NaN
2,2,0_0_10_10,2019-01-04 16:00:00,0,NaN
3,3,0_0_10_10,2019-01-04 17:00:00,0,NaN
4,4,0_0_10_10,2019-01-04 18:00:00,0,NaN


In [4]:
df_supply['timestamp'].max()

Timestamp('2020-02-04 23:00:00')

In [5]:
# Brief look at the supply data. Important note: there are some missing forecast values

# Forecast calculation:
# df['forecast']=df.groupby('zone').supply.shift(168)
df_supply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951758 entries, 0 to 951757
Data columns (total 5 columns):
Unnamed: 0    951758 non-null int64
zone          951758 non-null object
timestamp     951758 non-null datetime64[ns]
supply        951758 non-null int64
forecast      934958 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 36.3+ MB


### Feature engineering

In [6]:
# Feature ideas:
# holiday
# weekend
# friday_night
# holiday_week
# night
# airport_area
# city_center
# close_to_big_supply_area

In [7]:
# Create date and time columns out of timestamp. 
# Date will be later used to join df_supply and dataframe with features related to holidays and weather
df_supply['date'] = [d.date() for d in df_supply['timestamp']]
df_supply['time'] = [d.time() for d in df_supply['timestamp']]

In [8]:
df_supply['date'] = pd.to_datetime(df_supply['date'])

In [9]:
# Read Excel file with data about holidays and weather
file_name = "weather_holidays.xlsx"
my_sheet = 'ver2'

df_weather_holidays = read_excel(file_name, sheet_name = my_sheet, skiprows=1, parse_dates=['date'], index_col='date')
# df_weather_holidays.head()

In [10]:
# Brief look at the weather and holiday data. Note: there are some missing rain and temperature values.
df_weather_holidays.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2018-01-01 to 2020-12-31
Data columns (total 30 columns):
new_year                    1096 non-null int64
new_year_week               1096 non-null float64
epiphany                    1096 non-null int64
epiphany_week               1096 non-null float64
winter_holidays_1           1096 non-null int64
winter_holidays_2           1096 non-null int64
winter_holidays_3_warsaw    1096 non-null int64
winter_holidays_4           1096 non-null int64
easter                      1096 non-null int64
labour_day                  1096 non-null int64
constitution_day            1096 non-null int64
may_weel                    1096 non-null float64
corpus_christi              1096 non-null int64
corpus_christi_week         1096 non-null float64
assumption_of_mary          1096 non-null int64
assumption_of_mary_week     1096 non-null float64
all_saints_day              1096 non-null int64
all_saints_day_week         1096 non-null float64

In [11]:
# Merge holidays and weather dataframe with supply dataframe, to create extra features
df_all = pd.merge(df_supply,
                 df_weather_holidays,
                 on='date')
df_all.head()

,Unnamed: 0,zone,timestamp,supply,forecast,date,time,new_year,new_year_week,epiphany,...,christmas,christmas_week,new_years_eve,other,temperature_max,temperature_min,rain,sat_or_sun,holidays,holidays_week
0,0,0_0_10_10,2019-01-04 14:00:00,1,NaN,2019-01-04,14:00:00,0,0.5,0,...,0,0.0,0,0,0.0,-6.666667,0.508,0,0,0.5
1,1,0_0_10_10,2019-01-04 15:00:00,0,NaN,2019-01-04,15:00:00,0,0.5,0,...,0,0.0,0,0,0.0,-6.666667,0.508,0,0,0.5
2,2,0_0_10_10,2019-01-04 16:00:00,0,NaN,2019-01-04,16:00:00,0,0.5,0,...,0,0.0,0,0,0.0,-6.666667,0.508,0,0,0.5
3,3,0_0_10_10,2019-01-04 17:00:00,0,NaN,2019-01-04,17:00:00,0,0.5,0,...,0,0.0,0,0,0.0,-6.666667,0.508,0,0,0.5
4,4,0_0_10_10,2019-01-04 18:00:00,0,NaN,2019-01-04,18:00:00,0,0.5,0,...,0,0.0,0,0,0.0,-6.666667,0.508,0,0,0.5


In [12]:
def x_coordinate (row):
    return row['zone'][:1]

def y_coordinate (row):
    return row['zone'][2:3]

def weekday(row):
    return row['timestamp'].strftime('%A')

In [13]:
df_all['x_coordinate'] = df_all.apply (lambda row: x_coordinate(row), axis=1)

In [14]:
df_all['y_coordinate'] = df_all.apply (lambda row: y_coordinate(row), axis=1)

In [15]:
df_all['weekday'] = df_all.apply (lambda row: weekday(row), axis=1)

In [17]:
df_all.head()

,Unnamed: 0,zone,timestamp,supply,forecast,date,time,new_year,new_year_week,epiphany,...,other,temperature_max,temperature_min,rain,sat_or_sun,holidays,holidays_week,x_coordinate,y_coordinate,weekday
0,0,0_0_10_10,2019-01-04 14:00:00,1,NaN,2019-01-04,14:00:00,0,0.5,0,...,0,0.0,-6.666667,0.508,0,0,0.5,0,0,Friday
1,1,0_0_10_10,2019-01-04 15:00:00,0,NaN,2019-01-04,15:00:00,0,0.5,0,...,0,0.0,-6.666667,0.508,0,0,0.5,0,0,Friday
2,2,0_0_10_10,2019-01-04 16:00:00,0,NaN,2019-01-04,16:00:00,0,0.5,0,...,0,0.0,-6.666667,0.508,0,0,0.5,0,0,Friday
3,3,0_0_10_10,2019-01-04 17:00:00,0,NaN,2019-01-04,17:00:00,0,0.5,0,...,0,0.0,-6.666667,0.508,0,0,0.5,0,0,Friday
4,4,0_0_10_10,2019-01-04 18:00:00,0,NaN,2019-01-04,18:00:00,0,0.5,0,...,0,0.0,-6.666667,0.508,0,0,0.5,0,0,Friday


In [16]:
df_all.to_pickle("df_all.pkl")